In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
path = Path("cleaned_post_code_registration.csv")

In [ ]:
read0 = pd.read_csv(path)

In [ ]:
read0.head()

In [ ]:
read0.count()

In [ ]:
read0.dtypes


In [ ]:
read0["Fuel Type"].value_counts()

In [ ]:
read0["State"].value_counts()

In [ ]:
read0["Fuel Type"] = read0["Fuel Type"].replace({"BEV":"Battery Electric Vehicle",
                       'ICE':"Internal Combustion Engine",
                       'HFCEV':'Hydrogen Cell Vehicle',
                       'Hybrid/PHEV':'Hybrid'})

In [ ]:
read0.head()

In [ ]:
read1= read0.groupby(["Postcode","Fuel Type"])
read1

In [ ]:
read0.duplicated()